# TLS 1.3 handshake overview
This is the basic TLS 1.3 handshake:

<img src="images/handshake_tls13.png" alt="Handshake TLS 1.3" width="400"/>

## Dissecting the handshake

In [ ]:
from scapy.all import *
load_layer('tls')
conf.logLevel = logging.INFO

In [ ]:
# ClientHello
record1_str = open('raw_data/tls_session_13/01_cli.raw', 'rb').read()
record1 = TLS(record1_str)
sess = record1.tls_session
record1.show()

In [ ]:
# The PFS relies on the ECDH secret below being kept from observers, and deleted right after the key exchange
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey

# Used in records 2-6 + 8
x25519_client_privkey = open('raw_data/tls_session_13/cli_key.raw', 'rb').read()
sess.tls13_client_privshares["x25519"] = X25519PrivateKey.from_private_bytes(x25519_client_privkey)

# Used in records 7 + 9
x25519_server_privkey = open('raw_data/tls_session_13/srv_key.raw', 'rb').read()
sess.tls13_server_privshare["x25519"] = X25519PrivateKey.from_private_bytes(x25519_server_privkey)

In [ ]:
# ServerHello + ChangeCipherSpec (middlebox compatibility)
record2_str = open('raw_data/tls_session_13/02_srv.raw', 'rb').read()
record2 = TLS(record2_str, tls_session=sess.mirror())
record2.show()

In [ ]:
# Encrypted Extensions
record3_str = open('raw_data/tls_session_13/03_srv.raw', 'rb').read()
record3 = TLS(record3_str, tls_session=sess)
record3.show()

In [ ]:
# Certificate
record4_str = open('raw_data/tls_session_13/04_srv.raw', 'rb').read()
record4 = TLS(record4_str, tls_session=sess)
record4.show()

In [ ]:
# Certificate verify
record5_str = open('raw_data/tls_session_13/05_srv.raw', 'rb').read()
record5 = TLS(record5_str, tls_session=sess)
record5.show()

In [ ]:
# Finished
record6_str = open('raw_data/tls_session_13/06_srv.raw', 'rb').read()
record6 = TLS(record6_str, tls_session=sess)
record6.show()

In [ ]:
# Client ChangeCipherSpec (middlebox compatibility) + Finished
record7_str = open('raw_data/tls_session_13/07_cli.raw', 'rb').read()
record7 = TLS(record7_str, tls_session=sess.mirror())
record7.show()

## Dissecting some data

In [ ]:
# Client application data
record8_str = open('raw_data/tls_session_13/08_cli.raw', 'rb').read()
record8 = TLS(record8_str, tls_session=sess)
record8.show()

In [ ]:
# Server application data
record9_str = open('raw_data/tls_session_13/09_srv.raw', 'rb').read()
record9 = TLS(record9_str, tls_session=sess.mirror())
record9.show()

## Observations sur TLS 1.3
* Certificat désormais chiffré...
* ...mais pas le Server Name dans le ClientHello
* Risques du mode 0-RTT